#                         **Predict No of  upvotes**

An online question and answer platform has hired you as a data scientist to identify the best question authors on the platform. This identification will bring more insight into increasing the user engagement. Given the tag of the question, number of views received, number of answers, username and reputation of the question author, the problem requires you to predict the upvote count that the question will receive.

Data Dictionary

* Variables 
    * ID	: Question ID
    * Tag	: Anonymised tags representing question category
    * Reputation :	Reputation score of question author
    * Answers	: Number of times question has been answered
    * Username :	Anonymised user id of question author
    * Views :	Number of times question has been viewed
    * Upvotes	(Target)  : Number of upvotes for the question
 

Evaluation Metric : The evaluation metric for this competition is RMSE (root mean squared error)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns #importing seaborn module 
import warnings
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn import metrics
from pandas import set_option
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, cross_val_score # to split the data
from sklearn.metrics import explained_variance_score, median_absolute_error, r2_score, mean_squared_error, accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import xgboost as xgb
warnings.filterwarnings('ignore')

In [ ]:
#Load train and test files 
data = pd.read_csv('../input/predict-the-number-of-upvotes-a-post-will-get/train_NIR5Yl1.csv')
test = pd.read_csv('../input/predict-the-number-of-upvotes-a-post-will-get/test_8i3B3FC.csv')

In [ ]:
#creating dataframe for the required output
submission = pd.DataFrame()
submission['ID'] = test['ID']

In [ ]:
##Step 1 : Explore train and test datasets

#First look at train
data.sample(5)

In [ ]:
#First look at test
test.sample(5)

In [ ]:
#Shape of train and test
print('There are {} rows and {} columns in train'.format(data.shape[0],data.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
#Check Missing values in train
data.isna().sum()

In [ ]:
#Check Missing values in test
test.isna().sum()

No missing data in train and test

In [ ]:
#Check data types in train
data.info()

In [ ]:
#Check data types in test
test.info()

In [ ]:
#Lets describe train
data.describe()

In [ ]:
#Lets describe test
test.describe()

In [ ]:
#Lets concatenate train & test
df=pd.concat([data,test])
df.shape 

In [ ]:
#Lets drop ID column
df.drop('ID', axis = 1, inplace = True)

Identify categorical and numerical columns

In [ ]:
#Identify categorical columns 
df_cat=df.select_dtypes(include='object')
list(df_cat.columns)

In [ ]:
#Identify numerical columns
df_num=df.select_dtypes(include=['int64','float64'])
list(df_num.columns)

Step 1 : EDA (Exploratory Data Analysis)

In [ ]:
#Explore categorical variables - Tag
count = 1
for cols in df_cat:
    plt.subplot(1, 2, count)
    df[cols].value_counts().plot.pie(shadow=True,autopct='%1.1f%%',radius=1.1,textprops={'fontsize': 12} )
    count +=1
    plt.subplot(1, 2, count)
    plt.tight_layout()
    plt.style.use('seaborn-bright')
    df[cols].value_counts().plot.bar()
    fig=plt.gcf()
    fig.set_size_inches(12,5)
    plt.title('{0}'.format(cols))
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=12)
    plt.xticks(rotation=30)
    count+=1

* Variable : Tag
    * c & j : Maximum count of 100000+ (21.9%)
    * x : Minimum count of 9k+

In [ ]:
#Explore numerical variables distribution
sns.set(style='whitegrid', palette="plasma", font_scale=1.1, rc={"figure.figsize": [8, 5]})
df[list(df_num.columns)].hist(bins=20, figsize=(15, 6), layout=(2, 3));

For username : Max frequency is from 0 to 100000

In [ ]:
#Lets explore numerical variables : Answers
sns.set(style='whitegrid', palette="Spectral", font_scale=1.1, rc={"figure.figsize": [12, 5]})
sns.distplot(
    df['Answers'], norm_hist=False, kde=False, bins=50, hist_kws={"alpha": 1}
).set(xlabel='Answers', ylabel='Frequency');


Answers: Max frequency for answer count = 1 to 2

In [ ]:
# Lets identify which tag is having lowest/highest median value with respect to the numerical variables 
for cols in df_num:
    sorted_nb = df.groupby(['Tag'])[cols].median().sort_values()
    print("Sort numerical value by",sorted_nb)    

* Reputation : r(699) is min Vs C is max (1637)
* Answers  : Median is 2 for r&x Vs 3 for all others
* Username : It does not make any sense 
* Views    : r has min (2888) Vs j is max (14541) i.e. j tag has max views
* Upvotes : r has min (11) Vs j has max (42) median value

In [ ]:
#Lets visualize above details in boxen plot
#Numerical Data ['Reputation', 'Answers', 'Username', 'Views', 'Upvotes'] Vs Tag (i.e. categorical variable ) in ascending order of median 
plt.figure(figsize=(15, 15))
count = 1
for cols in df_num:
    sns.set(style='darkgrid', palette="Set1", font_scale=1.1, rc={"figure.figsize": [12, 12]})
    plt.subplot(3, 2, count)
    plt.tight_layout()
    sorted_nb = df.groupby(['Tag'])[cols].median().sort_values()
    sns.boxenplot(x=df['Tag'], y=df[cols], order=list(sorted_nb.index))
    plt.xticks(rotation=30)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=9)
    
    count+=1

Lets visualize regression plots between numerical variables Vs Target variable (i.e Upvotes)

In [ ]:
#Correlation between Views and Upvotes across each tag
sns.set(font_scale=1.0,style="ticks")
sns.lmplot(x="Views",y="Upvotes",data=data, col="Tag", col_wrap=5, ci=None, palette="muted", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Correlation between Answers and Upvotes across each tag
sns.set(font_scale=1.0,style="darkgrid")
sns.lmplot(x="Answers",y="Upvotes",data=data, col="Tag", col_wrap=5, ci=None, palette="plasma", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Correlation between Reputation and Upvotes across each tag
sns.set(font_scale=1.0,style="white")
sns.lmplot(x="Reputation",y="Upvotes",data=data, col="Tag", col_wrap=5, ci=None, palette="Set1", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Correlation between Views and Reputation across Tag
sns.set(font_scale=1.0,style="dark")
sns.lmplot(x="Views",y="Reputation",data=data, col="Tag", col_wrap=5, ci=None, palette="rocket", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Correlation between Answers and Reputation across Tag
sns.set(font_scale=1.0,style="dark")
sns.lmplot(x="Answers",y="Reputation",data=data, col="Tag", col_wrap=5, ci=None, palette="winter", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Correlation between Answers and Views across Tag
sns.set(font_scale=1.0,style="dark")
sns.lmplot(x="Answers",y="Views",data=data, col="Tag", col_wrap=5, ci=None, palette="summer", hue="Tag" , height=3, scatter_kws={"s": 10, "alpha": 1})
plt.show()

In [ ]:
#Lets do a correlation plot for entire dataframe
sns.heatmap(df.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':14})
fig=plt.gcf()
fig.set_size_inches(9,8)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

* Views are aving highest correlation of 0.44 followed by Reputation 0.27 and Answers 0.2 against Upvotes
* Avswers and Views are having high correlation of 0.5
* Username column has very low correation of -0.012 with Upvotes and can be dropped

Take backup of data

In [ ]:
df_backup = df.copy()

Feature Engineering

In [ ]:
#Tag -
Tag_dict = {'a':0, 'c':1,'h':2,'i':3,'j':4,'o':5,'p':6,'r':7,'s':8,'x':9}

# Transform categorical variable Tag to numbers
data["Tag"] = data["Tag"].apply(lambda x: Tag_dict[x])
test["Tag"] = test["Tag"].apply(lambda x: Tag_dict[x])

#Convert 'Tag' to object
data['Tag']  = data['Tag'].astype('object')
test['Tag']  = test['Tag'].astype('object')

In [ ]:
############ Creating a Count Column for Tag & Username caetgory #################
data["Tag_Count"] = data.groupby(['Tag'])['Tag'].transform('count')
Tag_Count_dict = data.groupby(['Tag']).size().to_dict()
test['Tag_Count'] = test['Tag'].apply(lambda x:Tag_Count_dict.get(x,0))


data["Username_Count"] = data.groupby(['Username'])['Username'].transform('count')
Username_Count_dict = data.groupby(['Username']).size().to_dict()
test['Username_Count'] = test['Username'].apply(lambda x:Username_Count_dict.get(x,0))


In [ ]:
##### Creating 25percentile and 75percentile columns on Upvotes feature  ########################

### Tag
Tag_25p_dict = data.groupby(['Tag'])['Upvotes'].apply(lambda x:np.percentile(x,25)).to_dict()
data['Tag_25PercPrice'] = data['Tag'].apply(lambda x:Tag_25p_dict.get(x,0))
test['Tag_25PercPrice'] = test['Tag'].apply(lambda x:Tag_25p_dict.get(x,0))

Tag_75p_dict = data.groupby(['Tag'])['Upvotes'].apply(lambda x:np.percentile(x,75)).to_dict()
data['Tag_75PercPrice'] = data['Tag'].apply(lambda x:Tag_75p_dict.get(x,0))
test['Tag_75PercPrice'] = test['Tag'].apply(lambda x:Tag_75p_dict.get(x,0))


In [ ]:
#Creating a Count Column for Views & Answers caetgory 
data["Views_Count"] = data.groupby(['Views'])['Views'].transform('count')
Views_Count_dict = data.groupby(['Views']).size().to_dict()
test['Views_Count'] = test['Views'].apply(lambda x:Views_Count_dict.get(x,0))

data["Answers_Count"] = data.groupby(['Answers'])['Answers'].transform('count')
Answers_Count_dict = data.groupby(['Answers']).size().to_dict()
test['Answers_Count'] = test['Answers'].apply(lambda x:Answers_Count_dict.get(x,0))

In [ ]:
#seperating the dependant variable 
train_y = data["Upvotes"]
data.drop(["Upvotes"], axis=1, inplace=True)
#Lets drop ID column
data.drop('ID', axis = 1, inplace = True)
test.drop('ID', axis = 1, inplace = True)

In [ ]:
#Create primary and secondary list of columns
primary_list   = ['Tag', 'Reputation', 'Answers', 'Username', 'Views','Tag_Count','Username_Count','Views_Count','Answers_Count']
secondary_list = ['Tag_25PercPrice', 'Tag_75PercPrice',]

In [ ]:
#model 1 dataframe
train1 = data[primary_list+secondary_list]
test1  = test[primary_list+secondary_list]

In [ ]:
#Shape of train and test
print('There are {} rows and {} columns in train'.format(train1.shape[0],train1.shape[1]))
print('There are {} rows and {} columns in test'.format(test1.shape[0],test1.shape[1]))


Model building starts

In [ ]:
#Creating train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train1.values, train_y.values, test_size = 0.2, random_state = 7)

After several permutaion and combination, I decided to use xgbregressor to predict which gave nest results

In [ ]:
#XGBoost Regressor
# Import XGBoost Regressor
from xgboost import XGBRegressor

#Create a XGBoost Regressor
reg = XGBRegressor(n_estimators=10000, learning_rate=0.05,)

# Train the model using the training sets 
reg.fit(X_train, y_train,early_stopping_rounds=5,eval_set=[(X_test, y_test)], verbose=0)

Prepare submission file

In [ ]:
predicted = reg.predict(test1.values)

#print(predicted.round())

submission['Upvotes'] = predicted.round()
submission.to_csv('./submission_rahulpednekar.csv',index=False)
submission.head()

> **Above prediction achieved rank 338 in analyticsVidhya hackathon on "Predict Number of Upvotes" **

> **If you have liked my Kernel then please upvote**